### Python Lab 2 Tutorial
This notebook will be an alternative the lecture notes as well as the coding section. It will be a mix of 3 components:
1. Going over the lecture notes and demonstrating the concepts within the context of our database.
    - This portion is done in the concepts notebooks
2. Setting up the tables(student/enroll/course) and showing the students the data
3. Covering the concepts in the coding example at the end of the lab. 


#### Imports
As with any python file, we first import the necessary modules and define global variables for connecting to the database.

In [1]:
import sqlite3
import time
import hashlib
import lab_utils as labUtils #contains some functions necessary for this lab
connection = None
cursor = None
dbPath="./register_code_example.db"

#### Initializing the database.
First, we want to initialize a database. For simplicity, the definition of tables, their initialization and insertion of data is abstracted away in the lab_utils file. While you do not need to be familiar with the contents of lab_utils for this tutorial, it is advised for students to take a quick look at the lab_utils file as a refresher or as a reference for future projects. 

In [2]:
#create database or overwrite it
labUtils.setupDatabase(dbPath)

created database at path: ./register_code_example.db


#### Connecting to the created database.
Next, we want to connec to the database to the created database, which has the name "register.db". You'll notice that "register.db" exists withing the same directory as lab_utils and the notebook you are reading currently. 

In [3]:
def connect(path):
    global connection, cursor
    connection = sqlite3.connect(path)
    cursor = connection.cursor()
    cursor.execute(' PRAGMA forteign_keys=ON; ')
    connection.commit()
    return connection,cursor

connect(dbPath)

(<sqlite3.Connection at 0x7f44d8d503b0>, <sqlite3.Cursor at 0x7f44d8d936c0>)

For convenience, we define a function that prints the contents of a table, given a table name. This function will be used throughout the tutorial to show contents of tables. For example printTable("students") would print the contents of the students table

In [4]:
def printTable(t):
    try:
        cursor.execute('select * from %s;'%(t,))
        tRows=cursor.fetchall()
        for row in tRows:
            print(row)
    except:
        print("couldn't print %s, are you sure it exists?"%(t,))

#### Getting familiar with the database
Let's first check the name of all tables in the database

In [5]:
tables=cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print("Our tables are",tables)
print("\nHere are the tables contents:")
for table in tables:
    t=table[0]
    print("\n",t,":\n")
    printTable(t)

Our tables are [('course',), ('student',), ('enroll',)]

Here are the tables contents:

 course :

(1, 'CMPUT 291', 197)
(2, 'CMPUT 391', 97)
(3, 'CMPUT 101', 297)

 student :

(1409106, 'Alex')
(1509106, 'Saeed')
(1609106, 'Mike')

 enroll :

(1409106, 1, '2019-09-11 14:36:03', 'A')
(1509106, 1, '2019-09-11 14:36:03', 'A')
(1609106, 1, '2019-09-11 14:36:03', 'C')
(1409106, 2, '2019-09-11 14:36:03', 'B')
(1509106, 2, '2019-09-11 14:36:03', 'C')
(1609106, 2, '2019-09-11 14:36:03', 'B')
(1409106, 3, '2019-09-11 14:36:03', 'F')
(1509106, 3, '2019-09-11 14:36:03', 'C')
(1609106, 3, '2019-09-11 14:36:03', None)


### Continuing the Example
#### We've already covered:
    - Schema:
            •course (course_id, title, seats_available)
            •student (student_id, name)
            •enroll (student_id, course_id, enroll_date, grade)
     - Our department offers some courses and we have a table for the students.
     - Every student can register in a course.
#### And now we want to implement the following:
     - A drop courses function that safely removes a student from a course given student and course ids.
     - A function that prints all students average numerical grade. 

### The drop function
One way to implement this function is to delete the entry in the enroll table that corresponds to the student's enrollment, then add one to the number of seats available for that course.
##### Notice that you can simply set seats_available=seats_available+1 in an SQLite query. 

In [6]:
def drop(student_id, course_id):
    global connection, cursor
    # Drop the course for the student 
    data = (student_id, course_id)    
    cursor.execute('DELETE FROM enroll WHERE student_id=? and course_id=?;', data)
    # Update the seats_avialable column
    data = (course_id, student_id, course_id)    
    cursor.execute( 
    ''' UPDATE course SET seats_available = seats_available + 1  
    where course_id=? and NOT EXISTS  (select * from enroll 
    WHERE student_id=? and course_id=?); ''' ,data)
    
    connection.commit()
    print("drop successeful!")
    return

### List of numerical student averages
Since student grades are stored as letter grades, we first define a python mapping function that maps letter grades to number, as discussed earlier in this tutorial. 

In [7]:
def GPA(grade):
    global connection, cursor
    # Map the grade to a numerical value
    if grade=='A':
        return 4   
    elif grade=='B':        
        return 3    
    elif grade=='C':       
        return 2
    return 0

We then use add this user defined function such that it can be utilized by SQLite

In [8]:
connection.create_function('GPA', 1, GPA)

Getting the AVG(grade) from our enroll table does not make any sense since grade is a TEXT data type. 
But AVG(GPA(grade)) does.

Now we can use the GPA function in a query to get the average numerical GPA. 


In [9]:
def getNumericAverages():   
    # We want a sorted list of the student names with their average GPAs.
    cursor.execute('''
    SELECT s.name, AVG(GPA(e.grade)) AS avg_gpa
    FROM student AS s, enroll AS e 
    WHERE s.student_id = e.student_id
    GROUP BY s.name
    ORDER BY avg_gpa;''') 
    all_entry = cursor.fetchall()    
    for one_entry in all_entry:        
        print (one_entry)   



Lets see if our function works:

In [10]:
getNumericAverages()

('Mike', 1.6666666666666667)
('Alex', 2.3333333333333335)
('Saeed', 2.6666666666666665)
